In [ ]:
#!jupyter nbconvert --to=python Chlsurf_OCCCI_Comparison_python3.ipynb

In [ ]:
class Chlsurf_OCCCI_comp:
    '''
    class Chlsurf_OCCCI_comp(runname,resultpath,savepath,meshpath,matfileChlsurf,first_year,last_year,
                 mapproj='pc',savefig=False,layerwise=False,n_levels = 2)
                 
    n_levels = 1: number of levels used for FESOM surface mean
    '''
    
    def __init__(self,runname,resultpath,savepath,meshpath,matfileChlsurf,first_year,last_year,
                 mapproj='pc',
                 savefig=False,
                 layerwise=False,depth_array=[],
                 n_levels = 1):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array
        self.n_levels = n_levels
        self.matfileChlsurf=matfileChlsurf
        

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        #from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle
        import scipy.io as spio


        import pyfesom2 as pf
        
        from plot_Taylor_normalized import plt_Taylor_norm
        
        if(self.mapproj == 'rob'): self.mapproj = ccrs.Robinson()
        elif(self.mapproj == 'pc'): self.mapproj = ccrs.PlateCarree()
        else: raise ValueError('Map projection not supported!')
        
        if(self.layerwise != False): raise ValueError('Layerwise processing not suported.')
        
        # load OCCCI CHl.a data -------------------------------------------------------------------------------------
        matChl = spio.loadmat(self.matfileChlsurf, squeeze_me=True)
        
        lat         = np.arange(-89.5,90.,1.)
        lon         = np.arange(-179.5,180.,1.)
        latdic, londic = np.meshgrid(lat, lon)
        
        #annualchl   = np.log10(matChl['x'])
        OCCCIchla = matChl['x']
        OCCCIchla_log10 = np.log10(OCCCIchla)
        
        OCCCIlabel = 'OCCCI: Surface Chl concentration'
        OCCCIunit = 'log10 Surface Chl.a [mg Chl m$^{-3}$]'
                
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = [self.fyear, self.lyear]
        
        lon_fesom = mesh.x2
        lat_fesom = mesh.y2        
        
        # load FESOM CHl.a data -------------------------------------------------------------------------------------
        #ncFESOMChl = resultpath + '/ChlA.fesom.1948.nc'
        #ncFESOMDiaChl = resultpath + '/DiaChl.fesom.1948.nc'
        #!ncdump -h $ncFESOMChl
        #!ncdump -h $ncFESOMDiaChl
        
        ChlAfesom = pf.get_data(resultpath, "ChlA", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        # surface: mean over n mesh levels
        #self.n_levels
        f_depth = mesh.zlev[self.n_levels]
        print('Using upper {0} layers to depth {1} m for surface FESOM data!'.format(
            n_levels,f_depth))
        
        ChlAfesom_surf = np.nanmean(ChlAfesom[:,:n_levels],axis=1)
        
        # mean over whole water column
        #ChlAfesom_mean = np.nanmean(ChlAfesom, axis=1)
        
        labelfesom = 'FESOM Chl.a {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'Chl.a [mmol m$^{-3}$]' 

        #DiaChlfesom = pf.get_data(resultpath, "DiaChl", years, mesh,how="mean", compute=True, runid=self.runname, silent=True)
        #DiaChlfesom_surf = DiaChlfesom[:,0]
        #DiaChlfesom_mean = np.mean(DiaChlfesom, axis=1)
        #labelfesomDia = 'FESOM Diatom Chl.a {0}-{1}'.format(self.fyear,self.lyear)
        
        # convert FESOM CHl.a data -------------------------------------------------------------------------------------
        #Chl.a : 893.509 g/mol
        conv = 893.509
        ChlAfesom_surf = ChlAfesom_surf * conv
        #ChlAfesom_mean = np.log10(ChlAfesom_mean * conv)
        #DiaChlfesom_surf = np.log10(DiaChlfesom_surf * conv)
        #DiaChlfesom_mean = np.log10(DiaChlfesom_mean * conv)
        unitfesom = 'FESOM Chl.a [mg m$^{-3}$]'
        
        # interpolate FESOM CHl.a to regular -------------------------------------------------------------------------------------
        ChlAfesom_surf_interp = pf.fesom2regular(
                data = ChlAfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        ChlAfesom_surf_interp_log10 = np.log10(ChlAfesom_surf_interp)
        
        if False: # interpolation check
            print('\nFESOM interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
                np.nanmin(ChlAfesom_surf),np.nanmax(ChlAfesom_surf),np.nanmean(ChlAfesom_surf),
                np.nanmin(ChlAfesom_surf_interp),np.nanmax(ChlAfesom_surf_interp),np.nanmean(ChlAfesom_surf_interp)))
        
            fig = plt.figure(figsize=(10,10))
            ax1 = plt.subplot(projection = ccrs.Robinson())

            m1 = plt.pcolormesh(londic, latdic, ChlAfesom_surf_interp_log10, 
                transform = ccrs.PlateCarree(),
                norm=colors.LogNorm(vmin=np.nanmin(ChlAfesom_surf_interp_log10), 
                                    vmax=np.nanmax(ChlAfesom_surf_interp_log10)),
                cmap='plasma')

            ax1.coastlines(resolution='110m', color='black', linewidth=1)

            cbar = fig.colorbar(m1,orientation = 'horizontal',fraction=0.1, pad=0.1) 
            cbar.set_label('Interpolated '+unitfesom, fontsize=20)
        
            
        # interpolate OCCCI CHl.a data -------------------------------------------------------------------------------------
        #OCCCIchla = griddata((londic.ravel(), latdic.ravel()), annualchl.ravel(), (lon_fesom, lat_fesom), method='nearest')
        #OCCCIchla = np.ma.filled(OCCCIchla, np.nan)

        # apply sea mask to OCCCI as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        OCCCIchla_ma = np.copy(OCCCIchla)
        OCCCIchla_ma[~np.isfinite(ChlAfesom_surf_interp)] = np.nan
        
        OCCCIchla_ma_log10 = np.log10(OCCCIchla_ma)
        
        # plot both CHl.a data -------------------------------------------------------------------------------------
        print('\nPlotting log10(Chl.a)\nOCCCI min = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f} (Mean over 0 to {4} m)'.format(
                np.nanmin(ChlAfesom_surf_interp_log10),np.nanmax(ChlAfesom_surf_interp_log10),
                np.nanmin(OCCCIchla_ma_log10),np.nanmax(OCCCIchla_ma_log10),
                f_depth))
        
        
        def mygrid(m):
            m.coastlines(resolution='110m', color='black', linewidth=1)
            
        fig, axes = plt.subplots(2,2, 
                                 subplot_kw=dict(projection=self.mapproj),
                                 gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                                 figsize=(20,17))
        # OC-CCI
        m1 = axes[0,0]
        f1 = m1.pcolormesh(londic, latdic, OCCCIchla_ma, 
                           transform = ccrs.PlateCarree(),
                            shading='flat', norm=colors.LogNorm(vmin=1e-3,vmax=5e3))
        mygrid(m1)
        m1.set_title('OC-CCI', fontsize=20)


        # FESOM
        m2 = axes[0,1]
        f2 = m2.pcolormesh(londic, latdic, ChlAfesom_surf_interp, 
                           transform = ccrs.PlateCarree(),
                            shading='flat',  norm=colors.LogNorm(vmin=1e-3,vmax=5e3))
        mygrid(m2)
        m2.set_title('{0}\n(0 to {1} m)'.format(labelfesom,f_depth), fontsize=20)

        # add one colorbar for first row plots below figure
        cbar = fig.colorbar(f1,
                            ax = axes[0,:2], 
                            location ='bottom',
                            fraction=0.1, pad=0.1) 
        cbar.ax.tick_params(labelsize=14)
        cbar.set_label('Chl.a [mg m$^{-3}$]', fontsize=20)
        
        # OC-CCI - FESOM
        m3 = axes[1,0]
        f3 = m3.pcolormesh(londic, latdic, OCCCIchla_ma_log10 - ChlAfesom_surf_interp_log10, 
                           transform = ccrs.PlateCarree(),
                            shading='flat', vmin=-5, vmax=5,
                           cmap='bwr')
        mygrid(m3)
        m3.set_title('OC-CCI - FESOM', fontsize=20)

        

        # add one colorbar for all plots below figure
        cbar = fig.colorbar(f3,
                            ax = axes[1,:1], 
                            location ='bottom',
                            fraction=0.1, pad=0.1) 
        cbar.ax.tick_params(labelsize=14)
        cbar.set_label('log10 Chl.a [mg m$^{-3}$]', fontsize=20)

        # fig export  -------------------------------------------------------------------------------------
        if(self.savefig==True):
            plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                    dpi = 300, bbox_inches='tight')
        plt.show(block=False)  

        
        # statistics  -------------------------------------------------------------------------------------            
        # preparation of datasets
        if np.isnan(np.min(OCCCIchla_ma_log10)): print('WARNING: OCCCI field contains NaNs')
        if np.isnan(np.min(ChlAfesom_surf_interp_log10)): print('WARNING: FESOM field contains NaNs')

        # get statistics only from valid OCCCI gridpoints 
        ind_stat = np.where(np.isfinite(OCCCIchla_ma_log10))
        
        title = 'log10 surface Chlorophyll'
        print('\nStatistics for '+title)
        plt_Taylor_norm(OCCCIchla_ma_log10[ind_stat],ChlAfesom_surf_interp_log10[ind_stat],
                                mask=True,title=title)


        # fig export  -------------------------------------------------------------------------------------
        if(self.savefig==True):                
            plt.savefig(self.savepath+self.runname+'_'+'Chla_OCCCI_Taylor'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                    dpi = 300, bbox_inches='tight')
        plt.show(block=False) 
        

In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo12'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'

    htmlname     =  runid + '.html'
    htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'

    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

    #modulepath = '/home/ollie/mozeisin/py_recom/GlobalAssessment'
    #modulepath   = '/home/ollie/jhauck/py_fesom/modules/'

    #obspath      = '/home/ollie/mseifert/py_fesom/CreateRunFiles/Initialization/Output/'

    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1970
    last_year  = 1980

    # OCCCI ------------------------------------------------------------------------------------  
    matfileChlsurf           = '/work/ollie/projects/MarESys/evaluation/climatology_annual_chl_1deg_OCCCI_2012_2015.mat'
    
    # now test:
    test = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,first_year,last_year,
                              mapproj = 'pc',
                              layerwise=False,savefig=False,
                             n_levels=1)